# TPF-03: Aprimoramentos do Modelo DistilBERT e Avaliação Final

## Introdução e Objetivos

Na etapa anterior TPF-02, foi reproduzido com sucesso o *fine-tuning* do modelo DistilBERT no dataset SST-2, alcançando uma acurácia base de ~90.37%.

O objetivo desta etapa TPF-03 é implementar e avaliar modificações arquiteturais e de processo de treinamento para otimizar o modelo. As modificações propostas visam **eficiência** e **convergência**:

1.  **Modificação 1 Arquitetura: Congelamento de Camadas Backbone Freezing.** Serão congelados os pesos dos *embeddings* e das camadas iniciais do Transformer 0 a 3, treinando apenas as camadas superiores 4 e 5 e o classificador. Isso visa reduzir o custo computacional e prevenir o esquecimento catastrófico.
2.  **Modificação 2 Treinamento: Scheduler Cosseno Cosine Annealing.** Substituição do decaimento linear padrão por um agendador de taxa de aprendizado cosseno, que proporciona uma convergência mais suave e potencialmente alcança mínimos locais mais robustos.

In [ ]:
# Força a instalação de versões específicas
!pip install -q --upgrade --force-reinstall numpy==1.26.4 pandas==2.2.2 pyarrow==15.0.2 \
datasets==2.20.0 transformers==4.44.2 evaluate==0.4.2 scikit-learn==1.5.1

# Reinício automático do Runtime para aplicar as mudanças
import os, time
print("Instalação concluída. Reiniciando o runtime em 3 segundos")
time.sleep(3)
os.kill(os.getpid(), 9)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

In [ ]:
# Imports e Configurações
import os
import torch
import numpy as np
import evaluate
import datasets
import transformers
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

# Configurações de Log
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

print(f"Transformers: {transformers.__version__}")

# VERIFICAÇÃO DE GPU
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"SUCESSO: GPU Encontrada: {gpu_name}")
else:
    print("FALHA: GPU NÃO ENCONTRADA!")

Transformers: 4.44.2
SUCESSO: GPU Encontrada: Tesla T4


## Preparação dos Dados

O dataset utilizado é o mesmo da etapa anterior (`glue/sst2`). O pré-processamento tokenização é mantido idêntico para garantir uma comparação justa dos resultados.

In [ ]:
# Checkpoint Original
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Carregar e Tokenizar
print("Carregando dataset 'glue/sst2'")
raw_datasets = load_dataset("glue", "sst2")

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

print("Tokenizando dados")
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Dados Prontos")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Tokenizando dados


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dados Prontos


## Implementação das Modificações

Nesta seção, será aplicada as alterações propostas:

1.  **Congelamento Freezing:** Iteração sobre os parâmetros do modelo. Definimos `requires_grad = False` para os *embeddings* e para as camadas 0, 1, 2 e 3 do DistilBERT. As camadas 4, 5 e o Classificador permanecem treináveis.
2.  **Scheduler:** No `TrainingArguments`, foi alterado `lr_scheduler_type` para `"cosine"` e aumentado levemente as épocas para 4, permitindo que o ciclo cosseno se complete adequadamente.

In [ ]:
# Carregar Métricas
metric = evaluate.load("accuracy")
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Carregar Modelo Original
print("Carregando modelo pré-treinado")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# APLICANDO MODIFICAÇÃO 1: CONGELAMENTO
print("\nAplicando Modificação 1: Congelamento de Camadas")

# Congela Embeddings
for param in model.distilbert.embeddings.parameters():
    param.requires_grad = False

# Congela as primeiras 4 camadas (índices 0, 1, 2, 3)
# O DistilBERT tem 6 camadas (0 a 5). Deixamos 4 e 5 livres.
for i, layer in enumerate(model.distilbert.transformer.layer):
    if i < 4:
        for param in layer.parameters():
            param.requires_grad = False
        print(f"-> Camada {i} congelada.")
    else:
        print(f"-> Camada {i} PERMANECE TREINÁVEL.")

# APLICANDO MODIFICAÇÃO 2: SCHEDULER COSSENO
print("\nAplicando Modificação 2: Configurando Scheduler Cosseno")

training_args = TrainingArguments(
    output_dir="distilbert_sst2_tpf03_melhorado",

    # Ajustes de Hiperparâmetros
    learning_rate=3e-5,       # Leve ajuste para compensar o congelamento
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,       # +1 época para o scheduler cosseno
    weight_decay=0.01,

    # A Modificação Principal aqui:
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,

    # Configurações Padrão
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Carregando modelo pré-treinado


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



Aplicando Modificação 1: Congelamento de Camadas
-> Camada 0 congelada.
-> Camada 1 congelada.
-> Camada 2 congelada.
-> Camada 3 congelada.
-> Camada 4 PERMANECE TREINÁVEL.
-> Camada 5 PERMANECE TREINÁVEL.

Aplicando Modificação 2: Configurando Scheduler Cosseno


## Execução e Resultados

Iniciamos o treinamento com as modificações ativas. Espera-se um tempo de treinamento por época menor devido ao congelamento e uma curva de aprendizado diferente devido ao scheduler.

In [ ]:
print("\nINICIANDO TREINAMENTO OTIMIZADO")
trainer.train()

print("\nAVALIAÇÃO FINAL")
eval_results = trainer.evaluate()
print(eval_results)


INICIANDO TREINAMENTO OTIMIZADO


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267900,0.290824,0.892202
2,0.142900,0.324195,0.888761
3,0.163000,0.382278,0.894495
4,0.168800,0.390543,0.895642



AVALIAÇÃO FINAL


{'eval_loss': 0.29082363843917847, 'eval_accuracy': 0.8922018348623854, 'eval_runtime': 1.2362, 'eval_samples_per_second': 705.364, 'eval_steps_per_second': 44.49, 'epoch': 4.0}


## Resultados Obtidos TPF-03

O treinamento foi realizado por 4 épocas utilizando a GPU T4. Abaixo, apresenta-se o histórico de desempenho registrado:

| Época | Training Loss | Validation Loss | Accuracy |
|:---:|:---:|:---:|:---:|
| **1** | 0.267900 | **0.290824** | **0.892202** |
| 2 | 0.142900 | 0.324195 | 0.888761 |
| 3 | 0.163000 | 0.382278 | 0.894495 |
| 4 | 0.168800 | 0.390543 | 0.895642 |

O modelo final selecionado pelo `Trainer` via `load_best_model_at_end` foi o da **Época 1**, pois apresentou a menor perda de validação 0.2908.

**Resultado Final:**
* **Eval Loss:** 0.2908
* **Eval Accuracy:** 89.22%
* **Tempo Total de Treino:** ~9 minutos

## Análise e Discussão

Nesta etapa, comparam-se os resultados do modelo aprimorado TPF-03 com a reprodução original TPF-02.

### Eficiência Computacional
A modificação mais impactante foi observada no tempo de treinamento.
* **TPF-02 CPU:** ~11 horas devido a limitações de hardware.
* **TPF-03 GPU + Congelamento:** ~9 minutos.
Além do uso da GPU, o congelamento das camadas iniciais 0 a 3 reduziu a quantidade de gradientes a serem calculados, tornando cada passo de treinamento mais leve.

### Trade-off: Desempenho vs. Rigidez
Observou-se uma leve queda na acurácia final:
* **TPF-02 Original:** 90.37%
* **TPF-03 Modificado:** 89.22%

Essa redução de aproximadamente **1.15%** é explicada pela técnica de *Freezing*. Ao congelar as camadas inferiores do Transformer, impediu-se que o modelo ajustasse suas representações linguísticas básicas ao vocabulário específico das críticas de filmes SST-2. O modelo tornou-se mais "rígido", porém muito mais eficiente.

### Convergência e Overfitting
A utilização do **Scheduler Cosseno** resultou em uma convergência extremamente rápida. O modelo atingiu seu pico de desempenho menor perda logo na **Época 1**.
Nas épocas subsequentes 2, 3 e 4, notou-se um aumento progressivo da `Validation Loss` de 0.29 para 0.39, indicando que as camadas treináveis classificador começaram a sofrer *overfitting* memorização do treino rapidamente. O mecanismo de *Early Stopping* foi crucial para descartar essas épocas degradadas e manter o melhor resultado.

## Conclusão

As modificações propostas no TPF-03 demonstraram um clássico *trade-off* de Engenharia de Machine Learning.
Trocou-se uma pequena fração de desempenho preditivo <1.5% por um modelo com significativamente menos parâmetros treináveis e convergência mais rápida.

O experimento validou que o DistilBERT possui representações pré-treinadas robustas o suficiente para obter ~89% de acurácia no SST-2 ajustando apenas suas camadas superiores, uma estratégia viável para cenários de recursos computacionais limitados.